# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import re
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

import pandas as pd
import numpy as np
from sqlalchemy import create_engine

from sklearn.metrics import confusion_matrix, classification_report
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.model_selection import train_test_split, GridSearchCV

import warnings
warnings.filterwarnings("ignore")

In [2]:
import _pickle as cPickle

In [58]:
nltk.download(['punkt','wordnet', 'stopwords'])

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ktmbugua\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\ktmbugua\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ktmbugua\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

In [3]:
# Directory locations
home = '../../'
data_dir = home + '/data'
raw_dir = data_dir + '/raw'
processed_dir = data_dir + '/processed'
modelling_dir = data_dir + '/for_modelling'

In [4]:
# load data from database
engine = create_engine(f'sqlite:///{processed_dir}/DisasterResponse.db')
df = pd.read_sql("select * from CategorisedMessages", engine)
X = df['message']
y = df.drop(columns=['message', 'id', 'original', 'genre'])

In [5]:
df

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26211,30261,The training demonstrated how to enhance micro...,None,news,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
26212,30262,A suitable candidate has been selected and OCH...,None,news,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
26213,30263,"Proshika, operating in Cox's Bazar municipalit...",None,news,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
26214,30264,"Some 2,000 women protesting against the conduc...",None,news,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


### 2. Write a tokenization function to process your text data

In [60]:
def tokenize(text):
    text = text.lower()
    punctuation_removed = re.sub(r'[^A-Za-z0-9]+', ' ', text)
    tokenized = word_tokenize(punctuation_removed)
    no_stop_words = [word for word in tokenized if word not in stopwords.words('english')] 
    
    lemmatizer = WordNetLemmatizer()
    clean_tokens = [lemmatizer.lemmatize(token).strip() for token in no_stop_words]
    
    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [72]:
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier()))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [68]:
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [73]:
%%time
pipeline.fit(X_train, y_train)

Pipeline(steps=[('vect',
                 CountVectorizer(tokenizer=<function tokenize at 0x000002814EE1B048>)),
                ('tfidf', TfidfTransformer()),
                ('clf',
                 MultiOutputClassifier(estimator=RandomForestClassifier()))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [74]:
y_pred = pipeline.predict(X_test)

In [ ]:
columns = y_test.columns
rows = []
for i in range(y_test.shape[1]):
    report = classification_report(y_test.values[:,i], y_pred[:,i], digits=3, output_dict=True)
    check_1 = report.get("1", None)
    output = dict(
            col_name=columns[i],
            accuracy=report["accuracy"],
            precision= None if check_1 is None else report["1"]["precision"],
            specificity=report["0"]["recall"],
            sensitivity= None if check_1 is None else report["1"]["recall"],
            f1score= None if check_1 is None else report["1"]["f1-score"],
        )
    rows.append(output)
    
performance = pd.DataFrame(rows)

In [111]:
performance

,col_name,accuracy,precision,specificity,sensitivity,f1score
0,related,0.821636,0.852834,0.452000,0.936179,0.892565
1,request,0.896247,0.825112,0.978489,0.495067,0.618834
2,offer,0.994965,0.000000,1.000000,0.000000,0.000000
3,aid_related,0.780592,0.746310,0.836125,0.699963,0.722394
4,medical_help,0.922643,0.612245,0.996852,0.057915,0.105820
5,medical_products,0.954532,0.789474,0.998717,0.093750,0.167598
6,search_and_rescue,0.974519,0.733333,0.999373,0.063218,0.116402
7,security,0.983827,0.500000,0.999690,0.018868,0.036364
8,military,0.973604,0.730769,0.998901,0.102703,0.180095
9,child_alone,1.000000,NaN,1.000000,NaN,NaN


In [ ]:
performance.accuracy.median(), performance.precision.median(), performance.f1score.median()

### 6. Improve your model
Use grid search to find better parameters. 

In [116]:
pipeline.get_params().keys()

dict_keys(['memory', 'steps', 'verbose', 'vect', 'tfidf', 'clf', 'vect__analyzer', 'vect__binary', 'vect__decode_error', 'vect__dtype', 'vect__encoding', 'vect__input', 'vect__lowercase', 'vect__max_df', 'vect__max_features', 'vect__min_df', 'vect__ngram_range', 'vect__preprocessor', 'vect__stop_words', 'vect__strip_accents', 'vect__token_pattern', 'vect__tokenizer', 'vect__vocabulary', 'tfidf__norm', 'tfidf__smooth_idf', 'tfidf__sublinear_tf', 'tfidf__use_idf', 'clf__estimator__bootstrap', 'clf__estimator__ccp_alpha', 'clf__estimator__class_weight', 'clf__estimator__criterion', 'clf__estimator__max_depth', 'clf__estimator__max_features', 'clf__estimator__max_leaf_nodes', 'clf__estimator__max_samples', 'clf__estimator__min_impurity_decrease', 'clf__estimator__min_impurity_split', 'clf__estimator__min_samples_leaf', 'clf__estimator__min_samples_split', 'clf__estimator__min_weight_fraction_leaf', 'clf__estimator__n_estimators', 'clf__estimator__n_jobs', 'clf__estimator__oob_score', 'clf_

In [117]:
%%time
parameters = {
    'vect__ngram_range': ((1, 1), (1, 2)),
    'vect__max_df': (0.5, 0.75, 1.0),
    'vect__max_features': (None, 5000, 10000),
    'clf__estimator__n_estimators': [50, 100, 200],
    'clf__estimator__min_samples_split': [2,3,4],
}

cv = GridSearchCV(pipeline, param_grid=parameters)

Wall time: 0 ns


In [ ]:
%%time
cv.fit(X_train, y_train)

In [ ]:
y_pred2 = cv.predict(X_test)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [ ]:
columns = y_test.columns
rows = []
for i in range(y_test.shape[1]):
    report = classification_report(y_test.values[:,i], y_pred2[:,i], digits=3, output_dict=True)
    check_1 = report.get("1", None)
    output = dict(
            col_name=columns[i],
            accuracy=report["accuracy"],
            precision= None if check_1 is None else report["1"]["precision"],
            specificity=report["0"]["recall"],
            sensitivity= None if check_1 is None else report["1"]["recall"],
            f1score= None if check_1 is None else report["1"]["f1-score"],
        )
    rows.append(output)
    
performance_cv = pd.DataFrame(rows)

In [ ]:
performance_cv

In [ ]:
performance_cv.accuracy.median(), performance_cv.precision.median(), performance_cv.f1score.median()

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

In [ ]:
with open(modelling_dir + '/model_output.pkl', mode='wb') as fp:
    cPickle.dump(cv, fp)

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.